An alternate approach to the one below [is shared on SO](https://stackoverflow.com/questions/12526606/callback-for-celery-apply-async)

Launch the celery worker

```bash
celery -A proj worker -c 2 -l INFO
```

For compile jobs, we can skip the concurrency argument and have as many prefork workers as there are cores, this way logic in Tuna would not have to deal with concurrency at all. 

For eval jobs, we will have to hardcode the concurrency to 1 since we want to attach each worker to a single GPU. This is required since I am not aware of how to get the concurrency ID of the worker. I could not find it documented yet.

```bash
# get num GPUs 
num_gpus=8
for gpu_id in 1 .. ${num_gpus}
do
    HIP_VISIBLE_DEVICES=${num_gpus} celery -A proj worker -c 1
done
```

To purge in flight tasks

``` bash
celery -A proj purge
```

Other useful commands are in the [worker guide](https://docs.celeryq.dev/en/stable/userguide/workers.html#queues)
We need to implement monitoring as well, to know how the tuning is going, details are in the [Monitoring and Management Guide](https://docs.celeryq.dev/en/stable/userguide/monitoring.html#guide-monitoring)


In [1]:
from celery.result import ResultSet
from proj.tasks import add
from proj.celery import app

In [ ]:
total_tasks = 200000
rs = ResultSet([])

def result_callback(task_id, value):
    result = app.AsyncResult(task_id).get()
    sum = result['sum']
    # print(f'{task_id} : done, {sum}')

for i in range(total_tasks):
    arg = {'x': i, 'y': i}
    # method without queues
    # rs.add(add.delay(arg))
    # queues for compile jobs and 
    # a separate queue for each GPU
    if i % 100 == 0:
        print(f'{i} jobs scheduled')
    rs.add(add.apply_async((arg,), queue='celery'))

print('All tasks added to queue')
_ = rs.join(callback=result_callback)


0 jobs scheduled
100 jobs scheduled
200 jobs scheduled
300 jobs scheduled
400 jobs scheduled
500 jobs scheduled
600 jobs scheduled
700 jobs scheduled
800 jobs scheduled
900 jobs scheduled
1000 jobs scheduled
1100 jobs scheduled
1200 jobs scheduled
1300 jobs scheduled
1400 jobs scheduled
1500 jobs scheduled
1600 jobs scheduled
1700 jobs scheduled
1800 jobs scheduled
1900 jobs scheduled
2000 jobs scheduled
2100 jobs scheduled
2200 jobs scheduled
2300 jobs scheduled
2400 jobs scheduled
2500 jobs scheduled
2600 jobs scheduled
2700 jobs scheduled
2800 jobs scheduled
2900 jobs scheduled
3000 jobs scheduled
3100 jobs scheduled
3200 jobs scheduled
3300 jobs scheduled
3400 jobs scheduled
3500 jobs scheduled
3600 jobs scheduled
3700 jobs scheduled
3800 jobs scheduled
3900 jobs scheduled
4000 jobs scheduled
4100 jobs scheduled
4200 jobs scheduled
4300 jobs scheduled
4400 jobs scheduled
4500 jobs scheduled
4600 jobs scheduled
4700 jobs scheduled
4800 jobs scheduled
4900 jobs scheduled
5000 jobs sc